In [1]:
# import 
# xml 형태로 데이터를 전송 
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from urllib import parse
import pymysql

In [2]:
url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000"

## 주소 분해 

In [4]:
result = parse.urlparse(url)

In [5]:
type(result)

urllib.parse.ParseResult

In [6]:
# 주소 출력 
result.geturl()

'http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

In [8]:
# hostname 출력하기 
result.hostname

'openapi.airport.co.kr'

In [9]:
# http ? https 
result.scheme

'http'

In [10]:
# 경로 출력
result.path

'/service/rest/FlightScheduleList/getIflightScheduleList'

In [11]:
# 쿼리 출력
result.query

'serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20151005&numOfRows=5000'

In [14]:
# 쿼리 분해 
query_dict = parse.parse_qs(result.query)
query_dict

{'serviceKey': ['WRvGBkn9UEtw+Asg0tYo210Etxvb1QEcAX+wfWvOxVGYJkh1CNZ/Y4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q=='],
 'schDate': ['20151005'],
 'numOfRows': ['5000']}

In [15]:
# 해당 api는 아래 url 참고 
# https://www.data.go.kr/data/15000126/openapi.do

In [16]:
query_dict['schDate'] = ['20220531']

In [17]:
query_dict

{'serviceKey': ['WRvGBkn9UEtw+Asg0tYo210Etxvb1QEcAX+wfWvOxVGYJkh1CNZ/Y4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q=='],
 'schDate': ['20220531'],
 'numOfRows': ['5000']}

In [18]:
parse.urlencode(query_dict, doseq = True)

'serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20220531&numOfRows=5000'

In [19]:
api_url = parse.ParseResult(scheme = result.scheme, netloc= result.hostname, path = result.path, 
                  params = result.params, query = parse.urlencode(query_dict, doseq = True), fragment=result.fragment)

In [21]:
url = parse.urlunparse(api_url)

In [22]:
for date_ in [str(x).replace("-", "")[:8] for x in pd.date_range('2022-01-01', '2022-05-31')][:10]:
    query_dict['schDate'] = [date_]
    api_url = parse.ParseResult(scheme = result.scheme, netloc= result.hostname, path = result.path, 
                  params = result.params, query = parse.urlencode(query_dict, doseq = True), fragment=result.fragment)
    url = parse.urlunparse(api_url)
    print(url)

http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20220101&numOfRows=5000
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20220102&numOfRows=5000
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20220103&numOfRows=5000
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=WRvGBkn9UEtw%2BAsg0tYo210Etxvb1QEcAX%2BwfWvOxVGYJkh1CNZ%2FY4QFa0r7j4bhT4NjPu7z1i1ck8ZgsKMt2Q%3D%3D&schDate=20220104&numOfRows=5000
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getIflightScheduleList?serviceKey=W

In [23]:
# xml 파싱
r = requests.get(url)

In [24]:
root = ET.fromstring(r.text)

In [25]:
root

<Element 'response' at 0x0000021047D91F40>

In [26]:
items = root.iter(tag='item')
for element in items:
    print("------------------------")
    air_dict = dict()
    for x in element:
        print(x.tag, x.text)

------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 싱가폴/싱가포르
airportCode SIN
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon Y
internationalNum KE645
internationalSat Y
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu Y
internationalTime 0010
internationalTue N
internationalWed N
------------------------
airlineEnglish KLM ROYAL DUTCH AIRLINES
airlineKorean 네덜란드항공
airport 암스테르담
airportCode AMS
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon Y
internationalNum KL856
internationalSat N
internationalStdate 2022-01-02T00:00:00+09:00
internationalSun Y
internationalThu N
internationalTime 0435
internationalTue N
internationalWed Y
------------------------
airlineEnglish ETIHAD AIRWAYS
airlineKorean 에티하드항공
airport 아부다비
airportCode AUH
city 인천
ci

airlineKorean 프랑스항공
airport 인천
airportCode ICN
city 파리/파리/샤를드골
cityCode CDG
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon Y
internationalNum AF264
internationalSat Y
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu Y
internationalTime 0810
internationalTue N
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 인천
airportCode ICN
city 방콕/방콕/수완나품
cityCode BKK
flightPurpose 여객기
internationalEddate 2022-02-01T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon Y
internationalNum KE654
internationalSat Y
internationalStdate 2022-01-02T00:00:00+09:00
internationalSun Y
internationalThu Y
internationalTime 0810
internationalTue Y
internationalWed Y
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 시드니
airportCode SYD
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-22T00:00:

airlineKorean 아시아나항공
airport 울란바토르/칭기스칸
airportCode UBN
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon N
internationalNum OZ567
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu N
internationalTime 1100
internationalTue N
internationalWed Y
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 타이페이/타이베이
airportCode TPE
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon N
internationalNum KE691
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun Y
internationalThu N
internationalTime 1100
internationalTue Y
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 오사카/간사이
airportCode KIX
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00

airlineEnglish JINAIR
airlineKorean 진에어
airport 인천
airportCode ICN
city 후쿠오카
cityCode FUK
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon Y
internationalNum LJ224
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu N
internationalTime 1330
internationalTue N
internationalWed N
------------------------
airlineEnglish ASIANA AIRLINE
airlineKorean 아시아나항공
airport 인천
airportCode ICN
city 나고야
cityCode NGO
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon N
internationalNum OZ121
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu N
internationalTime 1330
internationalTue N
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 로스앤젤레스
airportCode LAX
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2

city 암스테르담
cityCode AMS
flightPurpose 여객기
internationalEddate 2022-01-30T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon N
internationalNum KL855
internationalSat Y
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun Y
internationalThu Y
internationalTime 1540
internationalTue Y
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 인천
airportCode ICN
city 시애틀
cityCode SEA
flightPurpose 여객기
internationalEddate 2022-02-01T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon N
internationalNum KE020
internationalSat N
internationalStdate 2022-01-02T00:00:00+09:00
internationalSun Y
internationalThu Y
internationalTime 1545
internationalTue Y
internationalWed N
------------------------
airlineEnglish AIR INTERNATIONAL CHINA
airlineKorean 중국국제항공
airport 항조우/항저우
airportCode HGH
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationa

city 동경/도쿄/나리타
cityCode NRT
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri N
internationalIoType IN
internationalMon N
internationalNum KE704
internationalSat Y
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun Y
internationalThu Y
internationalTime 1730
internationalTue Y
internationalWed Y
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 인천
airportCode ICN
city 동경/도쿄/나리타
cityCode NRT
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon N
internationalNum KE704
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu N
internationalTime 1730
internationalTue N
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 인천
airportCode ICN
city 댈러스/댈러스/댈러스포트워스
cityCode DFW
flightPurpose 여객기
internationalEddate 2022-02-01T00:00:00+09:00
internationalFri N
internati

internationalWed Y
------------------------
airlineEnglish ASIANA AIRLINE
airlineKorean 아시아나항공
airport 시드니
airportCode SYD
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon N
internationalNum OZ6013
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun N
internationalThu N
internationalTime 2130
internationalTue N
internationalWed N
------------------------
airlineEnglish EVA AIRWAYS
airlineKorean 에바항공(장영항공)
airport 타이페이/타이베이
airportCode TPE
city 인천
cityCode ICN
flightPurpose 여객기
internationalEddate 2022-01-31T00:00:00+09:00
internationalFri Y
internationalIoType IN
internationalMon N
internationalNum BR159
internationalSat N
internationalStdate 2022-01-01T00:00:00+09:00
internationalSun Y
internationalThu N
internationalTime 2140
internationalTue N
internationalWed N
------------------------
airlineEnglish KOREAN AIR
airlineKorean 대한항공
airport 하노이
airportCode HAN
c

In [27]:
items = root.iter(tag='item')
total = list()
for element in items:
    air_dict = dict()
    # 하나의 items 정보가 dict형태로 저장 
    for x in element:
        air_dict[x.tag] = x.text
    total.append(air_dict)

In [30]:
def get_air_info(root):
    items = root.iter(tag='item')
    total = list()
    for element in items:
        air_dict = dict()
        # 하나의 items 정보가 dict형태로 저장 
        for x in element:
            air_dict[x.tag] = x.text
        total.append(air_dict)
    return pd.DataFrame(total)

In [33]:
# 진행상태 확인하는 모듈 
from tqdm import tqdm_notebook

In [34]:
air_total = []
for date_ in tqdm_notebook([str(x).replace("-", "")[:8] for x in pd.date_range('2022-01-01', '2022-05-31')]):
    query_dict['schDate'] = [date_]
    api_url = parse.ParseResult(scheme = result.scheme, netloc= result.hostname, path = result.path, 
                  params = result.params, query = parse.urlencode(query_dict, doseq = True), fragment=result.fragment)
    url = parse.urlunparse(api_url)
    r = requests.get(url)
    root = ET.fromstring(r.text)
    air_total.append(get_air_info(root))

C:\Users\82103\AppData\Local\Temp/ipykernel_17852/3954822953.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for date_ in tqdm_notebook([str(x).replace("-", "")[:8] for x in pd.date_range('2022-01-01', '2022-05-31')]):


  0%|          | 0/151 [00:00<?, ?it/s]

In [32]:
df = pd.concat(air_total) 

In [36]:
df.reset_index(drop=True, inplace=True)

In [38]:
df.isnull().sum()

airlineEnglish         0
airlineKorean          0
airport                0
airportCode            0
city                   0
cityCode               0
flightPurpose          0
internationalEddate    0
internationalFri       0
internationalIoType    0
internationalMon       0
internationalNum       0
internationalSat       0
internationalStdate    0
internationalSun       0
internationalThu       0
internationalTime      0
internationalTue       0
internationalWed       0
dtype: int64

In [40]:
(df == 'null').sum()

airlineEnglish         0
airlineKorean          0
airport                0
airportCode            0
city                   0
cityCode               0
flightPurpose          0
internationalEddate    0
internationalFri       0
internationalIoType    0
internationalMon       0
internationalNum       0
internationalSat       0
internationalStdate    0
internationalSun       0
internationalThu       0
internationalTime      0
internationalTue       0
internationalWed       0
dtype: int64

In [43]:
df[df == 'null'].notnull().sum()

airlineEnglish         0
airlineKorean          0
airport                0
airportCode            0
city                   0
cityCode               0
flightPurpose          0
internationalEddate    0
internationalFri       0
internationalIoType    0
internationalMon       0
internationalNum       0
internationalSat       0
internationalStdate    0
internationalSun       0
internationalThu       0
internationalTime      0
internationalTue       0
internationalWed       0
dtype: int64

In [44]:
# DataFrame에서 해당 값이 있는지 확인 (확인 차원!!!)
df2 = df.copy()
df2.iloc[0,0] = 'null'
df2[df2 == 'null'].notnull().sum()

airlineEnglish         1
airlineKorean          0
airport                0
airportCode            0
city                   0
cityCode               0
flightPurpose          0
internationalEddate    0
internationalFri       0
internationalIoType    0
internationalMon       0
internationalNum       0
internationalSat       0
internationalStdate    0
internationalSun       0
internationalThu       0
internationalTime      0
internationalTue       0
internationalWed       0
dtype: int64

In [47]:
from datetime import datetime

In [48]:
str(datetime.strptime("2021-12-02T00:00:00+09:00".split("+")[0], "%Y-%m-%dT%H:%M:%S"))

'2021-12-02 00:00:00'

In [45]:
" ".join("2021-12-02T00:00:00+09:00".split("+")[0].split("T"))

'2021-12-02 00:00:00'

In [49]:
# mysql에 저장할때 datetime
df['internationalEddate'] = df['internationalEddate'].apply(lambda x : " ".join(x.split("+")[0].split("T")) )

In [51]:
df.head()

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,KOREAN AIR,대한항공,싱가폴/싱가포르,SIN,인천,ICN,여객기,2022-01-31 00:00:00,N,IN,Y,KE645,Y,2022-01-01T00:00:00+09:00,N,Y,0010,N,N
1,ETIHAD AIRWAYS,에티하드항공,아부다비,AUH,인천,ICN,여객기,2022-01-31 00:00:00,Y,IN,N,EY857,Y,2022-01-01T00:00:00+09:00,N,N,0445,Y,Y
2,ETIHAD AIRWAYS,에티하드항공,아부다비,AUH,인천,ICN,여객기,2022-01-31 00:00:00,N,IN,Y,EY857,N,2022-01-01T00:00:00+09:00,Y,N,0445,N,N
3,ASIANA AIRLINE,아시아나항공,인천,ICN,마닐라,MNL,여객기,2022-01-01 00:00:00,Y,IN,Y,OZ704,Y,2021-12-02T00:00:00+09:00,Y,Y,0500,Y,Y
4,JEJU AIR,제주항공,인천,ICN,마닐라,MNL,여객기,2022-01-01 00:00:00,N,IN,N,7C2306,Y,2021-12-02T00:00:00+09:00,N,N,0500,N,N


In [53]:
df['internationalStdate'] = df['internationalStdate'].apply(lambda x : " ".join(x.split("+")[0].split("T")) )

In [54]:
df.head()

,airlineEnglish,airlineKorean,airport,airportCode,city,cityCode,flightPurpose,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed
0,KOREAN AIR,대한항공,싱가폴/싱가포르,SIN,인천,ICN,여객기,2022-01-31 00:00:00,N,IN,Y,KE645,Y,2022-01-01 00:00:00,N,Y,0010,N,N
1,ETIHAD AIRWAYS,에티하드항공,아부다비,AUH,인천,ICN,여객기,2022-01-31 00:00:00,Y,IN,N,EY857,Y,2022-01-01 00:00:00,N,N,0445,Y,Y
2,ETIHAD AIRWAYS,에티하드항공,아부다비,AUH,인천,ICN,여객기,2022-01-31 00:00:00,N,IN,Y,EY857,N,2022-01-01 00:00:00,Y,N,0445,N,N
3,ASIANA AIRLINE,아시아나항공,인천,ICN,마닐라,MNL,여객기,2022-01-01 00:00:00,Y,IN,Y,OZ704,Y,2021-12-02 00:00:00,Y,Y,0500,Y,Y
4,JEJU AIR,제주항공,인천,ICN,마닐라,MNL,여객기,2022-01-01 00:00:00,N,IN,N,7C2306,Y,2021-12-02 00:00:00,N,N,0500,N,N


In [ ]:
52.78.163.244

## db접속

In [58]:
# db 설치되어 있는 ip(namenode)
try:
    con = pymysql.connect(host='52.78.163.244', user='root', port=3306, password='12345', db='sqoop', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)

In [59]:
create_table = """
    CREATE TABLE airport_info (
       `airlineKorean`        VARCHAR(20),
        `airport`             VARCHAR(20), 
        `city`                VARCHAR(20),
        `internationalEddate` DATETIME,
        `internationalFri`    VARCHAR(1),
        `internationalIoType` VARCHAR(3),
        `internationalMon`    VARCHAR(1),
        `internationalNum`    VARCHAR(10),  
        `internationalSat`    VARCHAR(1),
        `internationalStdate` DATETIME,
        `internationalSun`    VARCHAR(1),
        `internationalThu`    VARCHAR(1),
        `internationalTime`   TIME,
        `internationalTue`    VARCHAR(1),
        `internationalWed`    VARCHAR(1),
        `year`                VARCHAR(4)
    ) COLLATE='utf8mb4_general_ci'
"""

In [60]:
cur.execute(create_table)
con.commit()

In [61]:
sql = """INSERT INTO airport_info (airlineKorean, airport, city, internationalEddate, internationalFri, internationalIoType, internationalMon, 
                              internationalNum, internationalSat, internationalStdate,
                              internationalSun, internationalThu, internationalTime, internationalTue, internationalWed, year) VALUES
         (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [62]:
df.columns

Index(['airlineEnglish', 'airlineKorean', 'airport', 'airportCode', 'city',
       'cityCode', 'flightPurpose', 'internationalEddate', 'internationalFri',
       'internationalIoType', 'internationalMon', 'internationalNum',
       'internationalSat', 'internationalStdate', 'internationalSun',
       'internationalThu', 'internationalTime', 'internationalTue',
       'internationalWed'],
      dtype='object')

In [63]:
df.drop(['airlineEnglish', 'airportCode', 'cityCode', 'flightPurpose'], axis=1, inplace=True)

In [65]:
df['year'] = '2022'

In [69]:
df.head(2)

,airlineKorean,airport,city,internationalEddate,internationalFri,internationalIoType,internationalMon,internationalNum,internationalSat,internationalStdate,internationalSun,internationalThu,internationalTime,internationalTue,internationalWed,year
0,대한항공,싱가폴/싱가포르,인천,2022-01-31 00:00:00,N,IN,Y,KE645,Y,2022-01-01 00:00:00,N,Y,0010,N,N,2022
1,에티하드항공,아부다비,인천,2022-01-31 00:00:00,Y,IN,N,EY857,Y,2022-01-01 00:00:00,N,N,0445,Y,Y,2022


In [71]:
# 피클로 만들기
# df.to_pickle("./data/220602airinfo/air_info.pkl")

In [78]:
df.to_pickle("./data/220602airinfo/air_info.pkl")

In [79]:
df = pd.read_pickle("./data/220602airinfo/air_info.pkl")
# Can't get attribute 에러 
# 강사님이 주신 피클을 read하려고하니 버전이 맞지않아서 에러가 뜬것같다.
# 내껄로 피클만들고 read함.

In [73]:
pd.__version__

'1.3.4'

In [74]:
pip install pandas --upgrade

  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'D:\\anaconda\\Lib\\site-packages\\~andas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [75]:
pd.__version__

'1.3.4'

In [80]:
for row in df.iterrows():
    try:
        cur.execute(sql, row[1].tolist())
    except Exception as e:
        print(e)
        break

In [81]:
con.commit()

In [ ]:
for row in tqdm_notebook(df.iterrows()):
    try:
        cur.execute(sql, row[1].tolist())
    except Exception as e:
        print(e)
        break